-   [Mass spectrometry and proteomics data
    analysis](#mass-spectrometry-and-proteomics-data-analysis)
    -   [Setup](#setup)
    -   [Introduction](#introduction)
    -   [Exploring available
        infrastructure](#exploring-available-infrastructure)
    -   [Mass spectrometry data](#mass-spectrometry-data)
    -   [Getting data from proteomics
        repositories](#getting-data-from-proteomics-repositories)
    -   [Handling raw MS data](#handling-raw-ms-data)
    -   [Handling identification data](#handling-identification-data)
    -   [MS/MS database search](#msms-database-search)
    -   [Analysing search results](#analysing-search-results)
    -   [High-level data interface](#high-level-data-interface)
    -   [Quantitative proteomics](#quantitative-proteomics)
    -   [Importing third-party quantitation
        data](#importing-third-party-quantitation-data)
    -   [Data processing and analysis](#data-processing-and-analysis)
        -   [Raw data processing](#raw-data-processing)
        -   [Processing and
            normalisation](#processing-and-normalisation)
    -   [Statistical analysis](#statistical-analysis)
        -   [Classification](#classification)
        -   [Clustering](#clustering)
    -   [Annotation](#annotation)
    -   [Other relevant
        packages/pipelines](#other-relevant-packagespipelines)
    -   [Session information](#session-information)

Mass spectrometry and proteomics data analysis
==============================================

Setup
-----

The follow packages will be used throughout this documents. R version `3.1.1` or higher is required to install all the packages using `BiocInstaller::biocLite`.

In [ ]:
library("mzR")
library("mzID")
library("MSnID")
library("MSnbase")
library("rpx")
library("MLInterfaces")
library("pRoloc")
library("pRolocdata")
library("MSGFplus")
library("rols")
library("hpar")

The most convenient way to install all the tutorials requirement (and more related content), is to install [`RforProteomics`](http://www.bioconductor.org/packages/release/data/experiment/html/RforProteomics.html) with all its dependencies.

``` r
library("BiocInstaller")
biocLite("RforProteomics", dependencies = TRUE)
```

Other packages of interest, such as [`rTANDEM`](http://www.bioconductor.org/packages/release/bioc/html/rTANDEM.html) or [`MSGFgui`](http://www.bioconductor.org/packages/release/bioc/html/MSGFgui.html) will be described later in the document but are not required to execute the code in this workflow.

Introduction
------------

This workflow illustrates R / Bioconductor infrastructure for proteomics. Topics covered focus on support for open community-driven formats for raw data and identification results, packages for peptide-spectrum matching, data processing and analysis:

-   Exploring available infrastructure
-   Mass spectrometry data
-   Getting data from proteomics repositories
-   Handling raw MS data
-   Handling identification data
-   MS/MS database search
-   Analysing search results
-   High-level data interface
-   Quantitative proteomics
-   Importing third-party quantitation data
-   Data processing and analysis
-   Statistical analysis
-   Machine learning
-   Annotation
-   Other relevant packages/pipelines

Links to other packages and references are also documented. In particular, the vignettes included in the [`RforProteomics`](http://www.bioconductor.org/packages/release/data/experiment/html/RforProteomics.html) package also contains relevant material.

Exploring available infrastructure
----------------------------------

In Bioconductor version 3.2, there are respectively 79 [proteomics](http://bioconductor.org/packages/release/BiocViews.html#___Proteomics), 55 [mass spectrometry software packages](http://bioconductor.org/packages/release/BiocViews.html#___MassSpectrometry) and 12 [mass spectrometry experiment packages](http://bioconductor.org/packages/release/BiocViews.html#___MassSpectrometryData). These respective packages can be extracted with the `proteomicsPackages()`, `massSpectrometryPackages()` and `massSpectrometryDataPackages()` and explored interactively.

``` r
library("RforProteomics")
pp <- proteomicsPackages()
display(pp)
```

Mass spectrometry data
----------------------

Most community-driven formats are supported in `R`, as detailed in the table below.

| Type           | Format                      | Package                                                                                                                                                       |
|:---------------|:----------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------|
| raw            | mzML, mzXML, netCDF, mzData | [`mzR`](http://bioconductor.org/packages/release/bioc/html/mzR.html) (read)                                                                                   |
| identification | mzIdentML                   | [`mzR`](http://bioconductor.org/packages/release/bioc/html/mzR.html) (read) and [`mzID`](http://bioconductor.org/packages/release/bioc/html/mzID.html) (read) |
| quantitation   | mzQuantML                   |                                                                                                                                                               |
| peak lists     | mgf                         | [`MSnbase`](http://bioconductor.org/packages/release/bioc/html/MSnbase.html) (read/write)                                                                     |
| other          | mzTab                       | [`MSnbase`](http://bioconductor.org/packages/release/bioc/html/MSnbase.html) (read/write)                                                                     |

Getting data from proteomics repositories
-----------------------------------------

MS-based proteomics data is disseminated through the [ProteomeXchange](http://www.proteomexchange.org/) infrastructure, which centrally coordinates submission, storage and dissemination through multiple data repositories, such as the [PRIDE](https://www.ebi.ac.uk/pride/archive/) data base at the EBI for MS/MS experiments, [PASSEL](http://www.peptideatlas.org/passel/) at the ISB for SRM data and the [MassIVE](http://massive.ucsd.edu/ProteoSAFe/static/massive.jsp) resource. The [`rpx`](http://www.bioconductor.org/packages/release/bioc/html/rpx.html) is an interface to ProteomeXchange and provides a basic access to PX data.

In [ ]:
library("rpx")
pxannounced()

Using the unique `PXD000001` identifier, we can retrieve the relevant metadata that will be stored in a `PXDataset` object. The names of the files available in this data can be retrieved with the `pxfiles` accessor function.

In [ ]:
px <- PXDataset("PXD000001")
px
pxfiles(px)

Other metadata for the `px` data set:

In [ ]:
pxtax(px)
pxurl(px)
pxref(px)

Data files can then be downloaded with the `pxget` function. Below, we retrieve the sixth file, TMT\_Erwinia\_1uLSike\_Top10HCD\_isol2\_45stepped\_60min\_01-20141210.mzML. The file is downloaded in the working directory and the name of the file is return by the function and stored in the `mzf` variable for later use.

In [ ]:
mzf <- pxget(px, pxfiles(px)[6])
mzf

Handling raw MS data
--------------------

The `mzR` package provides an interface to the [proteowizard](http://proteowizard.sourceforge.net/) C/C++ code base to access various raw data files, such as `mzML`, `mzXML`, `netCDF`, and `mzData`. The data is accessed on-disk, i.e it is not loaded entirely in memory by default but only when explicitly requested. The three main functions are `openMSfile` to create a file handle to a raw data file, `header` to extract metadata about the spectra contained in the file and `peaks` to extract one or multiple spectra of interest. Other functions such as `instrumentInfo`, or `runInfo` can be used to gather general information about a run.

Below, we access the raw data file downloaded in the previous section and open a file handle that will allow us to extract data and metadata of interest.

In [ ]:
library("mzR")
ms <- openMSfile(mzf)
ms

The `header` function returns the metadata of all available peaks:

In [ ]:
hd <- header(ms)
dim(hd)
names(hd)

We can extract metadata and scan data for scan 1000 as follows:

In [ ]:
hd[1000, ]
head(peaks(ms, 1000))
plot(peaks(ms, 1000), type = "h")

Below we reproduce the example from the `MSmap` function from the `MSnbase` package to plot a specific slice of the raw data using the `mzR` functions we have just described.

In [ ]:
## a set of spectra of interest: MS1 spectra eluted
## between 30 and 35 minutes retention time
ms1 <- which(hd$msLevel == 1)
rtsel <- hd$retentionTime[ms1] / 60 > 30 &
    hd$retentionTime[ms1] / 60 < 35

## the map
M <- MSmap(ms, ms1[rtsel], 521, 523, .005, hd)

plot(M, aspect = 1, allTicks = FALSE)

In [ ]:
plot3D(M)

In [ ]:
## With some MS2 spectra
i <- ms1[which(rtsel)][1]
j <- ms1[which(rtsel)][2]
M2 <- MSmap(ms, i:j, 100, 1000, 1, hd)
plot3D(M2)

Handling identification data
----------------------------

The `RforProteomics` package distributes a small identification result file (see `?TMT_Erwinia_1uLSike_Top10HCD_isol2_45stepped_60min_01.mzid`) that we load and parse using infrastructure from the [`mzID`](http://bioconductor.org/packages/release/bioc/html/mzID.html) package.

In [ ]:
library("mzID")
f <- dir(system.file("extdata", package = "RforProteomics"),
         pattern = "mzid", full.names=TRUE)
basename(f)
id <- mzID(f)
id

Various data can be extracted from the `mzID` object, using one the accessor functions such as `database`, `scans`, `peptides`, ... The object can also be converted into a `data.frame` using the `flatten` function.

The `mzR` package also provides support fast parsing `mzIdentML` files with the `openIDfile` function. As for raw data, the underlying C/C++ code comes from the [proteowizard](http://proteowizard.sourceforge.net/).

In [ ]:
library("mzR")
f <- dir(system.file("extdata", package = "RforProteomics"),
         pattern = "mzid", full.names=TRUE)

id1 <- openIDfile(f)
fid1 <- mzR::psms(id1)

head(fid1)

MS/MS database search
---------------------

While searches are generally performed using third-party software independently of R or can be started from R using a `system` call, the [`rTANDEM`](http://www.bioconductor.org/packages/release/bioc/html/rTANDEM.html) package allows one to execute such searches using the X!Tandem engine. The [`shinyTANDEM`](http://www.bioconductor.org/packages/release/bioc/html/shinyTANDEM.html) provides an experimental interactive interface to explore the search results.

``` r
library("rTANDEM")
?rtandem
library("shinyTANDEM")
?shinyTANDEM
```

Similarly, the [`MSGFplus`](http://www.bioconductor.org/packages/release/bioc/html/MSGFplus.html) package enables to perform a search using the MSGF+ engine, as illustrated below.

We search the TMT\_Erwinia\_1uLSike\_Top10HCD\_isol2\_45stepped\_60min\_01-20141210.mzML file against the fasta file from `PXD000001` using `MSGFplus`.

We first download the fasta files:

In [ ]:
fas <- pxget(px, pxfiles(px)[10])
basename(fas)

In [ ]:
library("MSGFplus")
msgfpar <- msgfPar(database = fas,
                   instrument = 'HighRes',
                   tda = TRUE,
                   enzyme = 'Trypsin',
                   protocol = 'iTRAQ')
idres <- runMSGF(msgfpar, mzf, memory=1000)
idres
## identification file (needed below)
basename(mzID::files(idres)$id)

(Note that in the `runMSGF` call above, I explicitly reduce the memory allocated to the java virtual machine to 3.5GB. In general, there is no need to specify this argument, unless you experience an error regarding the *maximum heap size*).

A graphical interface to perform the search the data and explore the results is also available:

``` r
library("MSGFgui")
MSGFgui()
```

Analysing search results
------------------------

The [`MSnID`](http://bioconductor.org/packages/release/bioc/html/MSnID.html) package can be used for post-search filtering of MS/MS identifications. One starts with the construction of an `MSnID` object that is populated with identification results that can be imported from a `data.frame` or from `mzIdenML` files.

In [ ]:
library("MSnID")
msnid <- MSnID(".")
msnid <- read_mzIDs(msnid,
                    basename(mzID::files(idres)$id))
show(msnid)

The package then enables to define, optimise and apply filtering based for example on missed cleavages, identification scores, precursor mass errors, etc. and assess PSM, peptide and protein FDR levels. Below, we start by apply a correction of monoisotopic peaks (see `?correct_peak_selection` for details) and define two variables to be used for identification filtering.

In [ ]:
msnid <- correct_peak_selection(msnid)
msnid$msmsScore <- -log10(msnid$`MS-GF:SpecEValue`)
msnid$absParentMassErrorPPM <- abs(mass_measurement_error(msnid))

As shown below, this particular spiked-in data set displays few high scoring non-decoy hits

We define a filter object, assigning arbitrary threshold and evaluate it on the `msnid` data

In [ ]:
filtObj <- MSnIDFilter(msnid)
filtObj$absParentMassErrorPPM <- list(comparison="<", threshold=5.0)
filtObj$msmsScore <- list(comparison=">", threshold=8.0)
filtObj

evaluate_filter(msnid, filtObj)

We can also optimise the filtering with a target protein FDR value of 0.01

In [ ]:
filtObj.grid <- optimize_filter(filtObj, msnid, fdr.max=0.01,
                                method="Grid", level="PSM",
                                n.iter=50000)
filtObj.grid
evaluate_filter(msnid, filtObj.grid)

We can now apply the filter to the data

In [ ]:
msnid <- apply_filter(msnid, filtObj.grid)
msnid

The resulting data can be exported to a `data.frame` or to a dedicated `MSnSet` data structure for quantitative MS data, described below, and further processed and analyses using appropriate statistical tests.

High-level data interface
-------------------------

The above sections introduced low-level interfaces to raw and identification results. The [`MSnbase`](http://bioconductor.org/packages/release/bioc/html/MSnbase.html) package provides abstractions for raw data through the `MSnExp` class and containers for quantification data via the `MSnSet` class. Both store

1.  the actual assay data (spectra or quantitation matrix, see below), accessed with `spectra` (or the `[`, `[[` operators) or `exprs`;
2.  sample metadata, accessed as a `data.frame` with `pData`;
3.  feature metadata, accessed as a `data.frame` with `fData`.

<!-- `]]`, `]` -->
Another useful slot is `processingData`, accessed with `processingData(.)`, that records all the processing that objects have undergone since their creation (see examples below).

The `readMSData` will parse the raw data, extract the MS2 spectra (by default) and construct an MS experiment object of class `MSnExp`.

(Note that while `readMSData` supports MS1 data, this is currently not convenient as all the data is read into memory.)

In [ ]:
library("MSnbase")
rawFile <- dir(system.file(package = "MSnbase", dir = "extdata"),
               full.name = TRUE, pattern = "mzXML$")
basename(rawFile)
msexp <- readMSData(rawFile, verbose = FALSE)
msexp

MS2 spectra can be extracted as a list of `Spectrum2` objects with the `spectra` accessor or as a subset of the original `MSnExp` data with the `[` operator. Individual spectra can be accessed with `[[`.

In [ ]:
length(msexp)
msexp[1:2]
msexp[[2]]

The identification results stemming from the same raw data file can then be used to add PSM matches.

In [ ]:
fData(msexp)
## find path to a mzIdentML file
identFile <- dir(system.file(package = "MSnbase", dir = "extdata"),
                 full.name = TRUE, pattern = "dummyiTRAQ.mzid")
basename(identFile)
msexp <- addIdentificationData(msexp, identFile)
fData(msexp)

The `readMSData` and `addIdentificationData` make use of `mzR` and `mzID` packages to access the raw and identification data.

Spectra and (parts of) experiments can be extracted and plotted.

In [ ]:
msexp[[1]]
plot(msexp[[1]], full=TRUE)

In [ ]:
msexp[1:3]
plot(msexp[1:3], full=TRUE)

Quantitative proteomics
-----------------------

There are a wide range of proteomics quantitation techniques that can broadly be classified as labelled vs. label-free, depending whether the features are labelled prior the MS acquisition and the MS level at which quantitation is inferred, namely MS1 or MS2.

|     | Label-free | Labelled   |
|-----|:-----------|:-----------|
| MS1 | XIC        | SILAC, 15N |
| MS2 | Counting   | iTRAQ, TMT |

In terms of raw data quantitation, most efforts have been devoted to MS2-level quantitation. Label-free XIC quantitation has however been addressed in the frame of metabolomics data processing by the [`xcms`](http://bioconductor.org/packages/release/bioc/html/xcms.html) infrastructure.

An `MSnExp` is converted to an `MSnSet` by the `quantitation` method. Below, we use the iTRAQ 4-plex isobaric tagging strategy (defined by the `iTRAQ4` parameter; other tags are available) and the `trapezoidation` method to calculate the area under the isobaric reporter peaks.

In [ ]:
plot(msexp[[1]], full=TRUE, reporters = iTRAQ4)

In [ ]:
msset <- quantify(msexp, method = "trap", reporters = iTRAQ4, verbose=FALSE)
exprs(msset)
processingData(msset)

Other MS2 quantitation methods available in `quantify` include the (normalised) spectral index `SI` and (normalised) spectral abundance factor `SAF` or simply a simple count method.

In [ ]:
exprs(si <- quantify(msexp, method = "SIn"))     
exprs(saf <- quantify(msexp, method = "NSAF"))

Note that spectra that have not been assigned any peptide (`NA`) or that match non-unique peptides (`npsm > 1`) are discarded in the counting process.

**See also** The [`isobar`](http://www.bioconductor.org/packages/release/bioc/html/isobar.html) package supports quantitation from centroided `mgf` peak lists or its own tab-separated files that can be generated from Mascot and Phenyx vendor files.

Importing third-party quantitation data
---------------------------------------

The PSI `mzTab` file format is aimed at providing a simpler (than XML formats) and more accessible file format to the wider community. It is composed of a key-value metadata section and peptide/protein/small molecule tabular sections.

In [ ]:
mztf <- pxget(px, pxfiles(px)[2])
(mzt <- readMzTabData(mztf, what = "PEP"))

It is also possible to import arbitrary spreadsheets as `MSnSet` objects into R with the `readMSnSet2` function. The main 2 arguments of the function are (1) a text-based spreadsheet and (2) column names of indices that identify the quantitation data. The latter can be queried with the `getEcols` function.

In [ ]:
csv <- dir(system.file ("extdata" , package = "pRolocdata"),
           full.names = TRUE, pattern = "pr800866n_si_004-rep1.csv")
getEcols(csv, split = ",")
ecols <- 7:10
res <- readMSnSet2(csv, ecols)
head(exprs(res))
head(fData(res))

Data processing and analysis
----------------------------

### Raw data processing

For raw data processing look at `MSnbase`'s `clean`, `smooth`, `pickPeaks`, `removePeaks` and `trimMz` for `MSnExp` and spectra processing methods.

The [`MALDIquant`](http://cran.r-project.org/web/packages/MALDIquant/index.html) and [`xcms`](http://bioconductor.org/packages/release/bioc/html/xcms.html) packages also features a wide range of raw data processing methods on their own ad hoc data instance types.

### Processing and normalisation

Each different types of quantitative data will require their own pre-processing and normalisation steps. Both `isobar` and `MSnbase` allow to correct for isobaric tag impurities normalise the quantitative data.

In [ ]:
data(itraqdata)
qnt <- quantify(itraqdata, method = "trap",
                reporters = iTRAQ4, verbose = FALSE)
impurities <- matrix(c(0.929,0.059,0.002,0.000,
                       0.020,0.923,0.056,0.001,
                       0.000,0.030,0.924,0.045,
                       0.000,0.001,0.040,0.923),
                     nrow=4, byrow = TRUE)
## or, using makeImpuritiesMatrix()
## impurities <- makeImpuritiesMatrix(4)
qnt.crct <- purityCorrect(qnt, impurities)
processingData(qnt.crct)

Various normalisation methods can be applied the `MSnSet` instances using the `normalise` method: variance stabilisation (`vsn`), quantile (`quantiles`), median or mean centring (`center.media` or `center.mean`), ...

In [ ]:
qnt.crct.nrm <- normalise(qnt.crct, "quantiles") 

The `combineFeatures` method combines spectra/peptides quantitation values into protein data. The grouping is defined by the `groupBy` parameter, which is generally taken from the feature metadata (protein accessions, for example).

In [ ]:
## arbitraty grouping
g <- factor(c(rep(1, 25), rep(2, 15), rep(3, 15)))
g
prt <- combineFeatures(qnt.crct.nrm, groupBy = g, fun = "sum")
processingData(prt)

Finally, proteomics data analysis is generally hampered by missing values. Missing data imputation is a sensitive operation whose success will be guided by many factors, such as degree and (non-)random nature of the missingness.

Below, missing values are randomly assigned to our test data and visualised on a heatmap.

In [ ]:
set.seed(1)
qnt0 <- qnt
exprs(qnt0)[sample(prod(dim(qnt0)), 10)] <- NA
table(is.na(qnt0))
image(qnt0)

Missing value in `MSnSet` instances can be filtered out and imputed using the `filterNA` and `impute` functions.

In [ ]:
## remove features with missing values
qnt00 <- filterNA(qnt0)
dim(qnt00)
any(is.na(qnt00))

In [ ]:
## impute missing values using knn imputation
qnt.imp <- impute(qnt0, method = "knn")
dim(qnt.imp)
any(is.na(qnt.imp))

There are various methods to perform data imputation, as described in `?impute`.

Statistical analysis
--------------------

R in general and Bioconductor in particular are well suited for the statistical analysis of data. Several packages provide dedicated resources for proteomics data:

-   [`MSstats`](http://www.bioconductor.org/packages/release/bioc/html/MSstats.html): A set of tools for statistical relative protein significance analysis in DDA, SRM and DIA experiments. Data stored in `data.frame` or `MSnSet` objects can be used as input.

-   [`msmsTest`](http://www.bioconductor.org/packages/release/bioc/html/msmsTests.html): Statistical tests for label-free LC-MS/MS data by spectral counts, to discover differentially expressed proteins between two biological conditions. Three tests are available: Poisson GLM regression, quasi-likelihood GLM regression, and the negative binomial of the [`edgeR`](http://www.bioconductor.org/packages/release/bioc/html/edgeR.html) package. All can be readily applied on `MSnSet` instances produced, for example by `MSnID`.

-   [`isobar`](http://www.bioconductor.org/packages/release/bioc/html/isobar.html) also provides dedicated infrastructure for the statistical analysis of isobaric data.

n\#\# Machine learning

The [`MLInterfaces`](http://www.bioconductor.org/packages/release/bioc/html/MLInterfaces.html) package provides a unified interface to a wide range of machine learning algorithms. Initially developed for microarray and `ExpressionSet` instances, the [`pRoloc`](http://www.bioconductor.org/packages/release/bioc/html/pRoloc.html) package enables application of these algorithms to `MSnSet` data.

### Classification

The example below uses `knn` with the 5 closest neighbours as an illustration to classify proteins of unknown sub-cellular localisation to one of 9 possible organelles.

In [ ]:
library("MLInterfaces")
library("pRoloc")
library("pRolocdata")
data(dunkley2006)
traininds <- which(fData(dunkley2006)$markers != "unknown")
ans <- MLearn(markers ~ ., data = t(dunkley2006), knnI(k = 5), traininds)
ans

### Clustering

#### kmeans

In [ ]:
kcl <- MLearn( ~ ., data = dunkley2006, kmeansI, centers = 12)
kcl
plot(kcl, exprs(dunkley2006))

A wide range of classification and clustering algorithms are also available, as described in the `?MLearn` documentation page. The `pRoloc` package also uses `MSnSet` instances as input and ,while being conceived with the analysis of spatial/organelle proteomics data in mind, is applicable many use cases.

Annotation
----------

All the [Bioconductor annotation infrastructure](http://bioconductor.org/help/workflows/annotation/annotation/), such as [`biomaRt`](http://bioconductor.org/packages/release/bioc/html/biomaRt.html), [`GO.db`](http://www.bioconductor.org/packages/release/data/annotation/html/GO.db.html), organism specific annotations, .. are directly relevant to the analysis of proteomics data. A total of 93 ontologies, including some proteomics-centred annotations such as the PSI Mass Spectrometry Ontology, Molecular Interaction (PSI MI 2.5) or Protein Modifications are available through the [`rols`](http://www.bioconductor.org/packages/release/bioc/html/rols.html).

In [ ]:
library("rols")
olsQuery("ESI", "MS")

Data from the [Human Protein Atlas](http://www.proteinatlas.org/) is available via the [`hpar`](http://www.bioconductor.org/packages/release/bioc/html/hpar.html) package.

Other relevant packages/pipelines
---------------------------------

-   Analysis of post translational modification with [`isobar`](http://www.bioconductor.org/packages/release/bioc/html/isobar.html).
-   Analysis of label-free data from a Synapt G2 (including ion mobility) with [`synapter`](http://www.bioconductor.org/packages/release/bioc/html/synapter.html).
-   Analysis of spatial proteomics data with [`pRoloc`](http://www.bioconductor.org/packages/release/bioc/html/pRoloc.html).
-   Analysis of MALDI data with the [`MALDIquant`](http://cran.r-project.org/web/packages/MALDIquant/index.html) package.
-   Access to the Proteomics Standard Initiative Common QUery InterfaCe with the [`PSICQUIC`](http://www.bioconductor.org/packages/release/bioc/html/PSICQUIC.html) package.

Additional relevant packages are described in the [`RforProteomics`](http://www.bioconductor.org/packages/release/data/experiment/html/RforProteomics.html) vignettes.

Session information
-------------------